## Scraping through the Hannover Messe 2018 web page
#### http://www.hannovermesse.de/
Befor starting checkout the permission on http://www.hannovermesse.de/robots.txt

In [131]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [132]:
url = 'http://www.hannovermesse.de/en/exhibitors/sitemap.xml'
res = requests.get(url)

##### Some information

In [133]:
for i in res.headers:
    print('{}: {}'.format(i,res.headers[i]))

Date: Sun, 07 Jan 2018 14:52:57 GMT
Server: Apache
Set-Cookie: JSESSIONID=519D821129F3943C07D88D612A35E118; Path=/; HttpOnly, SRV=b07|WlI0U; path=/
Transfer-Encoding: chunked
Content-Type: text/xml;charset=UTF-8
Cache-control: private


##### Get all exhibitor urls with BeatifulSoup

In [134]:
soup = bs(res.content, 'html.parser')
url_list = soup.find_all('loc')
df = pd.DataFrame([str(i).strip('loc></loc') for i in url_list])
print(df[0][0])
len(df)

http://www.hannovermesse.de/exhibitor/a-zeta-gomma/W673272


7213

## Get the data of one exhibitor

In [135]:
res = requests.get(df[0][0])
soup = bs(res.content, 'html.parser')

In [136]:
name = soup.find('h3', {'itemprop': 'companyName'})
name.text

'A ZETA GOMMA - FornitureIndustriali s.p.A.'

In [137]:
street = soup.find('div', {'itemprop':'street'})
street.text

'Via Radici in Piano 449/1'

In [138]:
city = soup.find('div', {'itemprop':'city'})
city.text

'41049 Sassuolo'

In [139]:
country = soup.find('div', {'itemprop':'country'})
country.text.split('\n')[0]

'Italy'

In [140]:
tel_number = soup.find('a', {'class':'tel-number'})
tel_number.text.strip('\t\n')

'+39 0536 867111'

In [141]:
fax = soup.find('div', {'itemprop':'fax'})
fax.text.split('Fax: ')[1]

'+39 0536 806884'

In [142]:
website = soup.find('a', {'class': 'textLink icon-external-link'})
website['href']

'http://www.azetagomma.com'

## Function to get data of all exhibitors

In [143]:
def checking(res):
    if res == None:
        res = 'NaN'
    else: pass
    return res
class TN():
    def __init__(self):
        self.text = '\tNo tel_number\n'

In [144]:
def get_data(df):
    
    data = {'name':[],
           'street':[],
           'city':[],
            'zip_code':[],
           'country':[],
           'tel_number':[],
           'website':[],}
    for n,i in enumerate(df[0]):
        res = requests.get(i)
        soup = bs(res.content, 'html.parser')
        
        name = soup.find('h3', {'itemprop': 'companyName'}).text
        street = soup.find('div', {'itemprop':'street'}).text
        city = soup.find('div', {'itemprop':'city'}).text
        country = soup.find('div', {'itemprop':'country'}).text.split('\n')[0]
        tel_number = [i.text.strip('\t\n') for i in [soup.find('a', {'class':'tel-number'}), TN()] if i != None][0]
        website = [i['href'] for i in [soup.find('a', {'class': 'textLink icon-external-link'}), 
                                       {'href': 'NaN'}] if i != None][0]
        data['name'].append(name)
        data['street'].append(street)
        data['city'].append(city.split(' ')[1])
        data['zip_code'].append(city.split(' ')[0])
        data['country'].append(country)
        data['tel_number'].append(tel_number)
        data['website'].append(website)

    ddf = pd.DataFrame(data)
    return ddf          

### Without multicore prossesing it takes more than 2 hours !!!!!

ddf = get_data(df)
ddf.head()

In [145]:
ddf.to_csv('exhibitors.csv')

118.33333333333333